In [1]:
import pandas as pd
import numpy as np

In [2]:
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [3]:
spark_session = SparkSession.builder\
        .master("spark://192.168.2.70:7077")\
        .appName("Jinglin_PartB")\
        .config("spark.dynamicAllocation.enabled", True)\
        .config("spark.dynamicAllocation.shuffleTracking.enabled", True)\
        .config("spark.shuffle.service.enabled", True)\
        .config("spark.dynamicAllocation.executorIdleTimeout", "30s")\
        .config("spark.cores.max", 4)\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

spark_context.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/22 12:40:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sqlContext = SQLContext(spark_session.sparkContext)

/home/ubuntu/.local/lib/python3.10/site-packages/pyspark/sql/context.py:77: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [5]:
#  Load the CSV file from HDFS, and call show() to verify the data is loaded correctly.
df = sqlContext.read.csv('hdfs://192.168.2.70:9000/parking-citations.csv',
                        header = 'true', inferSchema = 'true').cache()
df.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date| VIN|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount| Latitude|Longitude|Agency Description|Color Description|Body Style Description|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+---------+---------+------------------+-----------------+----------------------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    null|       null|            CA|         200304.0|null|HOND|        PA|   GY|

In [6]:
# Print the schema for the DataFrame
df.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- VIN: string (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [7]:
# Count the number of rows in the CSV file
df.count()

13077724

In [8]:
# Count the number of partitions in the underlying RDD.
df.rdd.getNumPartitions()

16

In [12]:
# Drop the columns VIN, Latitude and Longitude
df = df.drop("VIN", "Latitude", "Longitude")

In [14]:
df.printSchema()

root
 |-- Ticket number: string (nullable = true)
 |-- Issue Date: timestamp (nullable = true)
 |-- Issue time: double (nullable = true)
 |-- Meter Id: string (nullable = true)
 |-- Marked Time: double (nullable = true)
 |-- RP State Plate: string (nullable = true)
 |-- Plate Expiry Date: double (nullable = true)
 |-- Make: string (nullable = true)
 |-- Body Style: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Route: string (nullable = true)
 |-- Agency: double (nullable = true)
 |-- Violation code: string (nullable = true)
 |-- Violation Description: string (nullable = true)
 |-- Fine amount: double (nullable = true)
 |-- Agency Description: string (nullable = true)
 |-- Color Description: string (nullable = true)
 |-- Body Style Description: string (nullable = true)



In [25]:
# Find the maximum fine amount. 
df['Fine amount'].cast('float')
df.select(max(df['Fine amount']).alias("Fine_max")).show()

+--------+
|Fine_max|
+--------+
|  1100.0|
+--------+



In [30]:
# How many fines have this amount?
filterdata = df.filter(df['Fine amount'] == 1100.0)
occurrence = filterdata.count()
print(f'Fine_max shows {occurrence} times')

Fine_max shows 626 times


In [34]:
# Show the top 20 most frequent vehicle makes, and their frequencies
makes = df.groupby('Make').count()
makes.sort("count", ascending=False).take(20)

[Row(Make='TOYT', count=2150768),
 Row(Make='HOND', count=1479996),
 Row(Make='FORD', count=1116235),
 Row(Make='NISS', count=945133),
 Row(Make='CHEV', count=892676),
 Row(Make='BMW', count=603092),
 Row(Make='MERZ', count=543298),
 Row(Make='VOLK', count=432030),
 Row(Make='HYUN', count=404917),
 Row(Make='DODG', count=391686),
 Row(Make='LEXS', count=368420),
 Row(Make='KIA', count=328155),
 Row(Make='JEEP', count=316300),
 Row(Make='AUDI', count=255395),
 Row(Make='MAZD', count=242344),
 Row(Make='OTHR', count=205546),
 Row(Make='GMC', count=184889),
 Row(Make='INFI', count=174315),
 Row(Make='CHRY', count=159948),
 Row(Make='SUBA', count=154640)]

In [36]:
# Create a User Defined Function to create a new column 'color long'
def map_color(color):
    COLORS = {
        'AL': 'Aluminum',
        'AM': 'Amber',
        'BG': 'Beige',
        'BK': 'Black',
        'BL': 'Blue',
        'BN': 'Brown',
        'BR': 'Brown',
        'BZ': 'Bronze',
        'CH': 'Charcoal',
        'DK': 'Dark',
        'GD': 'Gold',
        'GO': 'Gold',
        'GN': 'Green',
        'GY': 'Gray',
        'GT': 'Granite',
        'IV': 'Ivory',
        'LT': 'Light',
        'OL': 'Olive',
        'OR': 'Orange',
        'MR': 'Maroon',
        'PK': 'Pink',
        'RD': 'Red',
        'RE': 'Red',
        'SI': 'Silver',
        'SL': 'Silver',
        'SM': 'Smoke',
        'TN': 'Tan',
        'VT': 'Violet',
        'WT': 'White',
        'WH': 'White',
        'YL': 'Yellow',
        'YE': 'Yellow',
        'UN': 'Unknown'
    }
    return COLORS.get(color, color)

convertUDF = udf(lambda color: map_color(color), StringType())
add_colorlong = df.withColumn("color long", convertUDF(col("Color")))
add_colorlong.show()

+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+----------+
|Ticket number|         Issue Date|Issue time|Meter Id|Marked Time|RP State Plate|Plate Expiry Date|Make|Body Style|Color|            Location|Route|Agency|Violation code|Violation Description|Fine amount|Agency Description|Color Description|Body Style Description|color long|
+-------------+-------------------+----------+--------+-----------+--------------+-----------------+----+----------+-----+--------------------+-----+------+--------------+---------------------+-----------+------------------+-----------------+----------------------+----------+
|   1103341116|2015-12-21 00:00:00|    1251.0|    null|       null|            CA|         200304.0|HOND|        PA|   GY|     13147 WELBY WAY|01521|   1.0|        4000A

In [53]:
df_istoyota = add_colorlong.withColumn("is_toyota", col("Make")=="TOYT")
color_count = df_istoyota.groupBy(["color long", "is_toyota"]).count()
toyota_color_count = color_count.filter(col("is_toyota")).orderBy(col("count").desc())
toyota_color_count.first()

Row(color long='Gray', is_toyota=True, count=489697)

In [ ]:
# Based on the result, gray is the most frequent color value for Toyotas.